In [ ]:
#
# PID Controller
#

import rospy
import sys

import time
import numpy as np
from std_msgs.msg import Int16, UInt8, Float32
import signal
import matplotlib.pyplot as plt

class pid_vel:
    
    def __init__(self, meters_per_second):
        # self.aimed_mps = meters_per_second # input value 
        self.aimed_tps = meters_per_second / ((5/9)/100) # 5/9 = 1 tick = 5/9cm
        
        self.current_speed = 0 # in Int16 values for topic /manual_control/speed Int16
        self.current_tps = 0
        
        self.ticks = 0 # 1m = ca 180 ticks 
        

        self.time_sum = 0 # runtime in seconds
    
        self.ticks_sub = rospy.Subscriber("/ticks", UInt8, self.callback_ticks, queue_size=20)
        self.speed_pub = rospy.Publisher("/manual_control/speed", Int16, queue_size=20)
        
        # for plotting
        self.time_vals = [0]
        self.velo_vals = [0]
        
    # calculate current speed at each tick 
    def callback_ticks(self, data):
        self.ticks += int(data.data)

    # use the counted ticks of the last second to compute velocity and execute PID control
    def pid_controller(self):
        print("test")
                
        K_P = 1 # to be adjusted by trial and error
        # small = slower oscillation and more time for I and D to work
        # <1 is advised
        
        K_I = 0.05 # to be adjusted by trial and error
        
        K_D = 0.9 # to be adjusted by trial and error
        
        time_sum = 0 # runtime in seconds        
        error_sum = 0
        
        error_prev = self.aimed_tps - self.current_tps
        ticks_prev = self.ticks
        
        while True:
            print("aimed_tps", self.aimed_tps,  "current_tps", self.current_tps)
            
            self.current_tps  = self.ticks - ticks_prev

            
            error = self.aimed_tps - self.current_tps
            error_sum += error * time_sum
            deriv = error - error_prev # discrete derivative (delta t = 1 second)
            
            speed_increase = K_P * error + K_D * deriv + K_I * error_sum  # P + I + D
            print("speed: ", self.current_speed, "speed increase: ", speed_increase)
            print("ticks: ", self.ticks, "\n")
            self.current_speed += speed_increase
            
            self.speed_pub.publish(np.int16(self.current_speed))

            
            error_prev = error
            ticks_prev = self.ticks
            time_sum += 1
            
            time.sleep(1 - (time.time() % 1)) 
"""           
            # for plotting
            self.time_vals.append(time_sum)
            self.velo_vals.append(self.current_speed)
            plt.plot(self.time_vals,self.velo_vals)
            plt.title("Velocity over Time, using PID-Control")
            plt.xlabel("time")
            plt.ylabel("velocity")
"""

        
def main(args):
    rospy.init_node('pid_velocity', anonymous=True)
    pidvel = pid_vel(0.6)
    pidvel.pid_controller()

    def rescue(*_):
        print(pid_vel.output)
        rospy.signal_shutdown("")

    signal.signal(signal.SIGINT, rescue)

    rospy.spin()
    

if __name__ == '__main__':
    main(sys.argv)

test
aimed_tps 107.99999999999999 current_tps 0
speed:  0 speed increase:  107.99999999999999
ticks:  0 

aimed_tps 107.99999999999999 current_tps 0
speed:  107.99999999999999 speed increase:  118.79999999999998
ticks:  0 

aimed_tps 107.99999999999999 current_tps 0
speed:  226.79999999999995 speed increase:  54.29999999999998
ticks:  41 

aimed_tps 107.99999999999999 current_tps 41
speed:  281.0999999999999 speed increase:  -3.10000000000003
ticks:  134 

aimed_tps 107.99999999999999 current_tps 93
speed:  277.9999999999999 speed increase:  47.39999999999997
ticks:  228 

aimed_tps 107.99999999999999 current_tps 94
speed:  325.39999999999986 speed increase:  26.49999999999996
ticks:  334 

aimed_tps 107.99999999999999 current_tps 106
speed:  351.8999999999998 speed increase:  0.9999999999999538
ticks:  455 

aimed_tps 107.99999999999999 current_tps 121
speed:  352.89999999999975 speed increase:  -5.00000000000006
ticks:  580 

aimed_tps 107.99999999999999 current_tps 125
speed:  347.8